In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive

#Mount Google Drive
drive.mount('/content/MyDrive/')

Mounted at /content/MyDrive/


In [4]:
# Define the path to the excel file (replace with your file path on Google Drive)
file_path = '/content/MyDrive/MyDrive/df3-filtered.csv'

In [5]:
# Read the .csv file into a DataFrame
df = pd.read_csv(file_path)

In [6]:
# Display the first few rows of the DataFrame
print(df.head())

              Datetime    TrafficV
0  2018-01-01 00:00:00   82.909091
1  2018-01-01 01:00:00   59.181818
2  2018-01-01 02:00:00  110.818182
3  2018-01-01 03:00:00  135.000000
4  2018-01-01 04:00:00  115.363636


In [7]:
# Calculate the percentage of missing values per column
missing_percentage = df.isnull().mean() * 100

# Print the missing percentage per column
print("Percentage of missing values per column:")
print(missing_percentage)

Percentage of missing values per column:
Datetime    0.0
TrafficV    0.0
dtype: float64


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Load dataset (ensure 'Datetime' is parsed correctly)
df = pd.read_csv(file_path, parse_dates=["Datetime"], index_col="Datetime")

# Feature engineering from 'Datetime'
df['hour'] = df.index.hour  # Extract hour from Datetime
df['day_of_week'] = df.index.dayofweek  # Extract day of the week
df['month'] = df.index.month  # Extract month

# Select features (using 'Datetime' features and TrafficV)
features = ["hour", "day_of_week", "month"]

# Create lag features for TrafficV (Using past 24 hours of TrafficV data)
for lag in range(1, 25):  # Create lag features for 24 previous hours of TrafficV
    df[f"TrafficV_lag{lag}"] = df["TrafficV"].shift(lag)

# Drop rows with missing values due to lag features
df = df.dropna()

# Define X (independent variables) and y (target variable)
X = df[features + [f"TrafficV_lag{lag}" for lag in range(1, 25)]]
y = df["TrafficV"]

# Standardize features (not target variable)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict using the trained model
y_pred = rf_model.predict(X_test)






In [9]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# plot predictions vs true values
plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='True Values', color='blue')
plt.plot(y_pred, label='Predicted Values', color='red')
plt.legend()
plt.title("Traffic Volume Prediction")
plt.xlabel("Sample")
plt.ylabel("Traffic Volume (TrafficV)")
plt.show()


Mean Absolute Error: 4.110983731416037
Mean Squared Error: 33.054629244030615
R-squared: 0.9664237988716


In [10]:
#scater splot
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.5, color="purple")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle="dashed", color="black")  # 45-degree reference line
plt.xlabel("Traffic Volume Prediction")
plt.ylabel("Predicted NO2")
plt.title("Actual vs. Predicted TrafficV Scatter Plot")
plt.show()